In [0]:
!pip install kaggle

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json

In [0]:
from google.colab import files
files.upload()

In [0]:
!ls -lha kaggle.json

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

In [0]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -p /content

In [0]:
import os
print(os.listdir('/content'))

In [0]:
!unzip \*.zip

In [0]:
print(os.listdir('/content'))

In [0]:
import pandas as pd
jigs = pd.read_csv('/content/train.csv')
train = jigs.copy()

In [0]:
train.head(10)

In [0]:
train.isnull().sum()

In [0]:
train.shape

In [0]:
sample = pd.read_csv('/content/sample_submission.csv')
sample.shape

In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, Adam, RMSprop

In [0]:
max_features = 20000
maxlen = 100
embed_size = 128

In [0]:
train_sentences = train["comment_text"].values
y = train["target"].values

In [0]:
jigs_test = pd.read_csv("/content/test.csv")
test = jigs_test.copy()
test_sentences = test["comment_text"].values

In [0]:
tokenizer = text.Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_sentences))
train_tokenized = tokenizer.texts_to_sequences(train_sentences)
test_tokenized = tokenizer.texts_to_sequences(test_sentences)
x_train = sequence.pad_sequences(train_tokenized, maxlen=maxlen)
x_test = sequence.pad_sequences(test_tokenized, maxlen=maxlen)

In [0]:
def get_model():
  inp = Input(shape=(maxlen, ))
  x = Embedding(max_features, embed_size)(inp)
  x = Bidirectional(LSTM(50, return_sequences=True))(x)
  x = GlobalMaxPool1D()(x)
  x = Dropout(0.1)(x)
  x = Dense(50, activation = "relu")(x)
  x = Dropout(0.1)(x)
  x = Dense(10, activation = "relu")(x)
  x = Dropout(0.1)(x)
  x = Dense(1, activation = "sigmoid")(x)
  model = Model(inputs=inp, outputs = x)
  model.compile(loss = 'mean_squared_error',
               optimizer = Adam(lr = 0.0001),
               metrics=['accuracy'])
  return model


In [0]:
model = get_model()
batch_size = 32
epochs = 2

file_path = "/content/weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early]
model.fit(x_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)


In [0]:
model.load_weights(file_path)
y_test = model.predict(x_test)

In [0]:
np.save("/content/drive/My Drive/jigsaw_test_pred.npy",y_test)

In [59]:
test.head(5)

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


In [60]:
sample.head(5)

,id,prediction
0,7000000,0.0
1,7000001,0.0
2,7000002,0.0
3,7000003,0.0
4,7000004,0.0


In [0]:
test["prediction"] = y_test

In [0]:
del test["comment_text"]

In [66]:
test.head(5)

,id,prediction
0,7000000,0.018963
1,7000001,0.008631
2,7000002,0.020944
3,7000003,0.042791
4,7000004,0.736264


In [0]:
test.to_csv("/content/drive/My Drive/jigsaw_pred.csv", index=False)